<center>
    <h1>The (almost) all-Python QEpy software</h1>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Pavanello Research Group</h1></p><p><h2>Rutgers University-Newark</h2></p></td>
  </tr>
  <tr>
    <td></td>
  </tr>
</table>

#### Quantum MultiScale School June 18th 2024

# Acknowledgements

<table>
  <tr>
      <td><p></p><p>Alina Umerbekova</p><p>Nicolas Viot</p><p>Xin Chen </p><p>Xuecheng Shao</p><p>Michele Pavanello</p><p>Valeria Rios </p><p>Jessica Martinez</p></td>
  </tr>
</table>

Funding: NSF, DOE, MolSSI, ACS-PRF, Rutgers

<center>
QEpy driver: Python driver class for Quantum ESPRESSO - KS-DFT in Python

<br>
<br>

<center>
    <div class="alert alert-success"> GOALS:</div>
    <div class="alert alert-success">1) Get Density of state and band structure of Graphene</div>
</center>

#### Getting started!

- Go to QEPy/eos in your tunneled Jupyter-Notebook

<br>
<center>
<h6> ./QEPy/band/QEPY-RISE.ipynb</h6>
</center>

- Run this alongside with me! 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qepy.calculator import QEpyCalculator
from qepy.io import QEInput

AttributeError: Please provide a library name

In [ ]:
from ase.io.trajectory import Trajectory
from ase.lattice.hexagonal import Graphene
from ase import Atoms
from ase.io import read,write
import py3Dmol

## Build your system

In [ ]:
atoms = Atoms(Graphene('C', latticeconstant={'a':2.46, 'c': 7}))
write('Graphene.xyz',atoms)
v = py3Dmol.view(data=open('./Graphene.xyz','r').read(), style='sphere')
v.addUnitCell()
v.zoomTo()
v.show()

## Build an 'Input' file

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'prefix': "'tmp'",
        'pseudo_dir': "'./'"},
    '&system': {
        'ibrav' : 0,
        'ecutwfc': 20,
        'ecutrho': 200},
    '&electrons': {
        'conv_thr' : 1.0e-8},
     'atomic_species': ['C 12.0107 C.pbe-rrkjus.UPF'],
     'k_points automatic': [' 9 9 1 0 0 0'],
}

## Download the Pseupotentials

In [ ]:
additional_files = ['https://pseudopotentials.quantum-espresso.org/upf_files/C.pbe-rrkjus.UPF']
from dftpy.formats import download_files
download_files(additional_files)

## Generate the calculator
##### Atom positions are update automatically by the calculator

In [ ]:
calc = QEpyCalculator(qe_options=qe_options, logfile='tmp.out')
atoms.calc=calc

## Run a ground state calculation

In [ ]:
energy = atoms.get_potential_energy()
efermi = calc.get_fermi_level() #Ry
print(efermi)

## DOS with a fine K-points

In [ ]:
qe_options['k_points automatic']= ['12 12 1 0 0 0']
energies, dos =  atoms.calc.get_dos(qe_options, width=0.6)

In [ ]:
emin = -16.0
emax = 8.0
plt.plot(energies, dos)
plt.xlim(emin, emax)
plt.axvline(x=0, ls='--')
plt.xlabel('Energy (Ry)')
plt.ylabel('DOS')

## Band structure with ASE bandpath

In [ ]:
lat = atoms.cell.get_bravais_lattice()
lat.plot_bz(show=True)
print(lat.description())

In [ ]:
path = atoms.cell.bandpath('GKMG',npoints=61)

In [ ]:
for kp in np.arange(atoms.calc.driver.get_number_of_k_points()):
    '''Irreducible k-points'''
    print(kp,atoms.calc.driver.get_occupation_numbers(kpt=kp))

In [ ]:
qe_options['&system']['nbnd']=6
band = atoms.calc.get_band_structure(qe_options, kpts=path, reference=efermi)

In [ ]:
bp=band.plot()
bp.set_ylim(-16,8)

## Plotting both Band structure and DOS 

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([.12, .07, .67, .85])

emin = -16.0
emax = 8.0

bands = band.subtract_reference()
bands.plot(ax=ax, emin=emin, emax=emax)

dosax = fig.add_axes([.8, .07, .17, .85])
dosax.plot(dos, energies)

dosax.set_ylim(emin, emax)
dosax.set_xlim(0, None)
dosax.set_yticks([])
dosax.set_xticks([])
dosax.set_xlabel("DOS", fontsize=18);

# Challenge

1. Obtain the Band structure and DOS for Silicon (Diamond, cubic cell, a,b,c = 0.543 nm) or your own system with QEPy.